# Bayesian Bandits: An Application of Thompson Sampling

This notebook implements the Thompson Sampling algorithm described in [Analysis of Thompson Sampling for the Multi-Armed Bandit Problem](http://proceedings.mlr.press/v23/agrawal12/agrawal12.pdf) by Shipra Agrawal and Navin Goyal and applies it to the Bayesian Bandit problem.

** Note that "bandit" and "arm" are used interchangeably in this notebook (and in the Online Learning literature in general). **

## Table of Contents
[Section 1: Thompson Sampling for Bernoulli Bandits](#section1)

[Section 2: Thompson Sampling for General Stochastic Bandits](#section2)

## Section 1: Thompson Sampling for Bernoulli Bandits <a name="section1"></a>

We will first examine the Bernoulli Bandit case where:
* Rewards $r_t \in \{0, 1\}$
* For arm $i$, the probability of success ($r_{t,i} = 1$) is its mean, $\mu_i$

We maintain a Bayesian prior on the arm means $\mu_i$. For Bernoulli rewards (either 0 or 1), it turns out that a Beta distribution is a convenient choice of priors. This is because if the prior is a Beta($\alpha$, $\beta$) distribution, then updating the posterior distributions become much simpler. After observing a Bernoulli trial, the posterior distribution is:
* Beta($\alpha+1$, $\beta$) if the trial succeeded (reward = 1)
* Beta($\alpha$, $\beta+1$) if the trial failed (reward = 0)

The Thompson Sampling algorithm initializes a uniform prior on all arms, meaning that arm $i$ has a prior Beta(1,1) on $\mu_i$ because Beta(1,1) is a uniform distribution on (0,1).

Define:
* $S_i(t)$ = number of successes (reward = 1) for arm $i$ up to time $t$
* $F_i(t)$ = number of failures (reward = 0) for arm $i$ up to time $t$

The algorithm will then update the distribution on $\mu_i$ as Beta($S_i(t)+1$, $F_i(t)+1$), sample from these posterior distributions, and play an arm according to the probability of its mean being the largest.

---
#### Algorithm 1: Thompson Sampling for Bernoulli Bandits
For each arm $i=1,...,N$, set $S_i=0$, $F_i=0$

For each  $t=1,2,...$ do

* For each arm $i=1,...,N$, sample $\theta_i(t)$ from the Beta($S_i+1$, $F_i+1$) distribution
* Play arm $i(t) := argmax_i$ $\theta_i(t)$ and observe reward $r_t$
* If $r_t=1$, then $S_i(t) = S_i(t)+1$, else $F_i(t) = F_i(t)+1$
---

In [1]:
import numpy as np

In [24]:
class BernoulliBandit(object):
    def __init__(self, num_bandits):
        """num_bandits must be at least 2"""
        
        assert num_bandits >= 2, "Number of bandits must be at least 2"
        
        # Initialize the means for each of the bandits randomly
        self.bandits = np.random.random_sample(num_bandits)
        self.num_bandits = len(self.bandits)
        # Keep track of the largest probability to use in regret calculation as "best bandit"
        self.max_prob = np.max(self.bandits)
    
    def chooseBandit(self, bandit_num):
        """
        Chooses the indicated bandit and returns True if we get a reward, False otherwise.
        """
        # Index out of range
        assert bandit_num >= 0 and bandit_num < self.num_bandits, \
            "Index Out of Range: {}. There are {} bandits.".format(bandit_num, self.num_bandits)
            
        # Sample from Bernoulli distribution with probability of success equal to the bandit mean
        return np.random.binomial(1, self.bandits[bandit_num]) == 1
    
    def computeRegret(self, bandit_num):
        """
        Computes the regret accrued by the indicated bandit. By definition, this is how much reward we lost by
        choosing the indicated bandit instead of the best one (indicated by the bandit with the largest mean).
        """
        # Index out of range
        assert bandit_num >= 0 and bandit_num < self.num_bandits, \
            "Index Out of Range: {}. There are {} bandits.".format(bandit_num, self.num_bandits)
        
        # Difference in expected reward for choosing a suboptimal bandit
        return self.max_prob - self.bandits[bandit_num]

    
def draw_bandit_distribution(stats):
    """
    Samples the probability of each bandit yielding a reward given the number of successes and failures so far.
    This is computed using the Beta distribution.
    
    stats = [{'num_wins': __, 'num_losses': __}, ...]
    len(stats) should be equal to the number of bandits
    """
    reward_probs = np.zeros(len(stats))
    
    for bandit_num in range(len(stats)):
        num_wins, num_losses = stats[bandit_num]['num_wins'], stats[bandit_num]['num_losses']
        # The alpha parameter is num_wins+1 and the beta parameter is num_losses+1
        reward_probs[bandit_num] = np.random.beta(num_wins+1, num_losses+1)
    
    return reward_probs

## Section 2: Thompson Sampling for General Stochastic Bandits <a name="section2"></a>